In [1]:
import sys
import pm4py
from datetime import datetime
import numpy as np

In [2]:
sys.path.append("../utils/")
from runner import Runner

In [3]:
runner = Runner("Matrix-Chain", "../Matrix-Chain-4/variants-linnea/generate-variants-linnea.py")
args = ["10","110","120","130","500"]
ret = runner.generate_run_experiments(args,bRun=True, bGenerate=False)
ct,et,rt = runner.get_all_tables(runner.get_exp_dir(args), meta=True)

Running Experiments
Experiments completed


In [4]:
def get_trace_durations(event_table):
    dfs= event_table.drop_duplicates('case:concept:name', keep='first')[['case:concept:name','timestamp:start']]
    dfe= event_table.drop_duplicates('case:concept:name', keep='last')[['case:concept:name','timestamp:end']]
    dfm = dfs.merge(dfe, on='case:concept:name')
    dfm['case:duration'] = dfm.apply(lambda row: row['timestamp:end']-row['timestamp:start'], axis=1)
    dfm = dfm.rename(columns={
        'timestamp:start':'case:timestamp:start',
        'timestamp:end':'case:timestamp:end',
    })
    return dfm

In [5]:
case_durations = get_trace_durations(rt)

In [6]:
case_table = case_durations.merge(ct, on="case:concept:name")

In [7]:
case_table

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
0,algorithm1,1.654773e+09,1.654773e+09,0.004089,14900000.0,3
1,algorithm5,1.654773e+09,1.654773e+09,0.001164,29100000.0,3
2,algorithm4,1.654773e+09,1.654773e+09,0.000550,29100000.0,3
3,algorithm0,1.654773e+09,1.654773e+09,0.000480,2710000.0,3
4,algorithm3,1.654773e+09,1.654773e+09,0.000597,18000000.0,3
5,algorithm2,1.654773e+09,1.654773e+09,0.000910,17900000.0,3


In [8]:
worst_flops = case_table[case_table['case:flops']==case_table['case:flops'].max()]
worst_flops

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
1,algorithm5,1.654773e+09,1.654773e+09,0.001164,29100000.0,3
2,algorithm4,1.654773e+09,1.654773e+09,0.000550,29100000.0,3


In [9]:
best_duration = case_table[case_table['case:duration']==case_table['case:duration'].min()]
best_duration

,case:concept:name,case:timestamp:start,case:timestamp:end,case:duration,case:flops,case:num_kernels
3,algorithm0,1.654773e+09,1.654773e+09,0.00048,2710000.0,3


In [10]:
'algorithm4' in worst_flops['case:concept:name'].values

True

In [11]:
filtered_df = rt.loc[rt['case:concept:name'].isin(worst_flops['case:concept:name'].values)]

In [12]:
filtered_df

,case:concept:name,concept:name,concept:kernel,timestamp:start,timestamp:end
3,algorithm5,gemm_2.64e+05,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml4)",1.654773e+09,1.654773e+09
4,algorithm5,gemm_1.56e+07,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml5)",1.654773e+09,1.654773e+09
5,algorithm5,gemm_1.32e+07,"gemm!('N', 'N', 1.0, ml4, ml5, 0.0, ml6)",1.654773e+09,1.654773e+09
6,algorithm4,gemm_1.56e+07,"gemm!('N', 'N', 1.0, ml2, ml3, 0.0, ml4)",1.654773e+09,1.654773e+09
7,algorithm4,gemm_2.64e+05,"gemm!('N', 'N', 1.0, ml0, ml1, 0.0, ml5)",1.654773e+09,1.654773e+09
8,algorithm4,gemm_1.32e+07,"gemm!('N', 'N', 1.0, ml5, ml4, 0.0, ml6)",1.654773e+09,1.654773e+09


In [13]:
filtered_df = rt.loc[rt['case:concept:name'].isin(best_duration['case:concept:name'].values)]

In [14]:
filtered_df

,case:concept:name,concept:name,concept:kernel,timestamp:start,timestamp:end
9,algorithm0,gemm_3.12e+05,"gemm!('N', 'N', 1.0, ml1, ml2, 0.0, ml4)",1.654773e+09,1.654773e+09
10,algorithm0,gemm_1.3e+06,"gemm!('N', 'N', 1.0, ml4, ml3, 0.0, ml5)",1.654773e+09,1.654773e+09
11,algorithm0,gemm_1.1e+06,"gemm!('N', 'N', 1.0, ml0, ml5, 0.0, ml6)",1.654773e+09,1.654773e+09
